In [ ]:
!pip install git+https://github.com/eyurtsev/kor

In [31]:
from ctransformers import AutoModelForCausalLM

llm = AutoModelForCausalLM.from_pretrained(
    "/home/ubuntu/models/starchat-beta-ggml-q4_1.bin",
    model_type="starcoder",
    lib="/home/ubuntu/ctransformers/lib/libctransformers.so",
    local_files_only=True,
)

def inference(prompt: str):
    tokens = ""
    for token in llm(
        prompt,
        stream=True,
        threads=16,
        top_k=50,
        top_p=0.95,
        temperature=0.2,
        repetition_penalty=1.2,
        reset=True,
        max_new_tokens=2048,
    ):
        print(token, end="", flush=True)
        tokens += token

In [33]:
"""Fake ChatModel for testing purposes."""
from typing import Any, List, Mapping, Optional

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.chat_models.base import SimpleChatModel
from langchain.schema import BaseMessage


class LocalStarChatModel(SimpleChatModel):

    @property
    def _llm_type(self) -> str:
        return "starchat-beta"

    def _call(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        """First try to lookup in queries, else return 'foo' or 'bar'."""
        
        # <|system|>\n{system}<|end|>\n<|user|>\n{query}<|end|>\n<|assistant|>
        query = ""
        for message in messages:
            match message.type:
                case "system":
                    query += f"<|system|>\n{message.content}<|end|>\n"
                case "human":
                    query += f"<|user|>\n{message.content}<|end|>\n"
                case "ai":
                    query += f"<|assistant|>\n{message.content}<|end|>\n"
        query += "<|assistant|>"
        return inference(query)

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {}


In [36]:
from langchain.chat_models import ChatOpenAI
from kor import create_extraction_chain, Object, Text

model = LocalStarChatModel()

# llm = ChatOpenAI(
#     model_name="gpt-3.5-turbo",
#     temperature=0,
#     max_tokens=2000,
#     frequency_penalty=0,
#     presence_penalty=0,
#     top_p=1.0,
# )

schema = Object(
    id="player",
    description=(
        "User is controlling a music player to select songs, pause or start them or play"
        " music by a particular artist."
    ),
    attributes=[
        Text(
            id="song",
            description="User wants to play this song",
            examples=[],
            many=True,
        ),
        Text(
            id="album",
            description="User wants to play this album",
            examples=[],
            many=True,
        ),
        Text(
            id="artist",
            description="Music by the given artist",
            examples=[("Songs by paul simon", "paul simon")],
            many=True,
        ),
        Text(
            id="action",
            description="Action to take one of: `play`, `stop`, `next`, `previous`.",
            examples=[
                ("Please stop the music", "stop"),
                ("play something", "play"),
                ("play a song", "play"),
                ("next song", "next"),
            ],
        ),
    ],
    many=False,
)

chain = create_extraction_chain(model, schema, encoder_or_encoder_class='json')
chain.predict_and_parse(text="play songs by paul simon and led zeppelin and the doors")['data']

# {'player': {'artist': ['paul simon', 'led zeppelin', 'the doors']}}


I'm sorry, but I am unable to fulfill your request. Please provide a valid command in the form of JSON following the template provided above.

ValidationError: 1 validation error for AIMessage
content
  none is not an allowed value (type=type_error.none.not_allowed)